In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import json
import re
pd.set_option('display.max_rows', 500)

In [55]:
def getWikipediaList(url, table_index):
    
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all('table')
    table = tables[table_index]
    table_headers = []
    row_headers = []
    for tx in table.find_all('th'):
        if tx.has_attr('scope'):
            if tx['scope'] == 'col':
                table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
            if tx['scope'] == 'row':
                row_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
        else:
            table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
    result = pd.DataFrame([[td.text for td in row.find_all('td')] for row in table.tbody.find_all('tr')])
    print(row_headers)
    result.columns = table_headers
    result.replace(r'\n', '', regex = True, inplace = True)
    result.replace(r'[^\x00-\x7F]+', '', regex = True, inplace = True)
    result.drop([0], inplace= True)
    if(len(row_headers)>0):
        result = result.shift(periods=1, axis=1)

        result = result[0:len(result)-1]
        result['Name'] = row_headers
    return result

In [147]:
interpol_url = 'https://en.wikipedia.org/wiki/Interpol'
page = requests.get(interpol_url)
soup = bs4.BeautifulSoup(page.content, 'html.parser')
interpol_countries = []

data = soup.find("div", class_="plainlist").find("ul")
text = list(data.descendants)


for country in text:
    element = country.find('a')
    if isinstance(element, bs4.element.Tag):
        interpol_countries.append(element)
    #countries[-1] = countries[-1].text
    

for i in range(0, len(interpol_countries)):
    interpol_countries[i] = interpol_countries[i].text
interpol_countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Republic of the Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea

In [3]:
# THIS CELL IS FOR DEBUGGING THE ABOVE FUYNCTION
url = 'https://en.wikipedia.org/wiki/OECD'
page = requests.get(url)
soup = bs4.BeautifulSoup(page.content, 'html.parser')
tables=soup.find_all('table')
table = tables[11]
table_headers = []
row_headers = []
#print(table)
for tx in table.find_all('th'):
    if tx.has_attr('scope'): #and tx['scope'] == 'col':
        if tx['scope'] == 'row':
            row_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
        else:
            table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
    else:
        table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))


print(table_headers)
result = pd.DataFrame([[td.text for td in row.find_all('td')] for row in table.tbody.find_all('tr')])
result.columns = table_headers
result.replace(r'\s', '', regex = True, inplace = True)
result.drop([0], inplace= True)
result = result.shift(periods=1, axis=1, )
result = result[0:len(result)-1]
#print(len(row_headers))

['Country', 'Application', 'Negotiations', 'Invitation', 'Membership', 'Geographic location', 'Notes']


In [54]:
###### 
# UNFCC
# EU
# NATO
# OECD
# G7
# UN
# OSCE
# COE
# ILO
# INTERPOL
uncfcc_countries = getWikipediaList(url = 'https://en.wikipedia.org/wiki/List_of_parties_to_the_United_Nations_Framework_Convention_on_Climate_Change', table_index=0)
eu_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_state_of_the_European_Union', table_index=1)
nato_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_states_of_NATO', 0)
oecd_countries = getWikipediaList('https://en.wikipedia.org/wiki/OECD', 11)
g7_countries = getWikipediaList('https://en.wikipedia.org/wiki/Group_of_Seven', 4)
un_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_states_of_the_United_Nations', 1)
osce_countries = getWikipediaList('https://en.wikipedia.org/wiki/Organization_for_Security_and_Co-operation_in_Europe', 1)
coe_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_states_of_the_Council_of_Europe', 0)
# eu standardizationz
eu_countries = eu_countries.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
eu_countries = eu_countries.apply(lambda x: x.str.replace("\[.\]", "") if x.dtype == "object" else x)
#un standardization
un_countries = un_countries.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
un_countries = un_countries.apply(lambda x: x.str.replace("\[.*\]", "") if x.dtype == "object" else x)
#osce standardization
germany_index = osce_countries[osce_countries['State'] == 'Germany- as West Germany- as East Germany'].index[0]
north_macedonia = osce_countries[osce_countries['State'] == 'North Macedonia[Note 3][9]'].index[0]
russia_index = osce_countries[osce_countries['State'] == 'Russia (as Soviet Union)'].index[0]
serbia_index = osce_countries[osce_countries['State'] == 'Serbia (as Yugoslavia)'].index[0]

osce_countries.at[germany_index, 'State'] = 'Germany'
osce_countries.at[north_macedonia, 'State'] = 'Macedonia [FYROM]'
osce_countries.at[russia_index, 'State'] = 'Russia'
osce_countries.at[serbia_index, 'State'] = 'Serbia'

[]
[' Austria', ' Belgium', ' Bulgaria', ' Croatia', ' Cyprus[a]', ' Czech Republic', ' Denmark[d]', ' Estonia', ' Finland[e]', ' France[f]', ' Germany', ' Greece', ' Hungary', ' Ireland', ' Italy', ' Latvia', ' Lithuania', ' Luxembourg', ' Malta', ' Netherlands[i]', ' Poland', ' Portugal[k]', ' Romania', ' Slovakia', ' Slovenia', ' Spain[l]', ' Sweden']
[]
[]
[]
[]
[]
[]


In [149]:
coordinates = pd.read_csv('assets/world_country_and_usa_states_latitude_and_longitude_values.csv')
coordinates = coordinates[['country_code', 'latitude', 'longitude', 'country']]
coordinates['in_eu'] = np.where(coordinates['country'].isin(eu_countries['Name'].values), True, False)
coordinates['in_uncfcc'] = np.where(coordinates['country'].isin(uncfcc_countries['State'].values), True, False)
coordinates['in_nato'] = np.where(coordinates['country'].isin(nato_countries['Member state'].values), True, False)
coordinates['in_oecd'] = np.where(coordinates['country'].isin(oecd_countries['Country'].values), True, False)
coordinates['in_g7'] = np.where(coordinates['country'].isin(g7_countries['Member'].values), True, False)
coordinates['in_un'] = np.where(coordinates['country'].isin(un_countries['Member state'].values), True, False)
coordinates['in_osce'] = np.where(coordinates['country'].isin(osce_countries['State'].values), True, False)
coordinates['in_coe'] = np.where(coordinates['country'].isin(coe_countries['State'].values), True, False)
coordinates['in_ilo'] = np.where(coordinates['country'].isin(un_countries['Member state'].values), True, False)
coordinates['in_interpol'] = np.where(coordinates['country'].isin(interpol_countries), True, False)
not_in_ilo = ['Andorra', 'Bhutan', 'Liechtenstein', 'Micronesia', 'Monaco', 'Nauru', 'North Korea']

for country in not_in_ilo:
    coordinates.loc[coordinates['country'] == country, 'in_ilo'] = False 

coordinates

,country_code,latitude,longitude,country,in_eu,in_uncfcc,in_nato,in_oecd,in_g7,in_un,in_osce,in_coe,in_ilo,in_interpol
0,AD,42.546245,1.601554,Andorra,False,True,False,False,False,True,True,True,False,True
1,AE,23.424076,53.847818,United Arab Emirates,False,True,False,False,False,True,False,False,True,True
2,AF,33.939110,67.709953,Afghanistan,False,True,False,False,False,True,False,False,True,True
3,AG,17.060816,-61.796428,Antigua and Barbuda,False,True,False,False,False,True,False,False,True,True
4,AI,18.220554,-63.068615,Anguilla,False,False,False,False,False,False,False,False,False,False
5,AL,41.153332,20.168331,Albania,False,True,True,False,False,True,True,True,True,True
6,AM,40.069099,45.038189,Armenia,False,True,False,False,False,True,True,True,True,True
7,AN,12.226079,-69.060087,Netherlands Antilles,False,False,False,False,False,False,False,False,False,False
8,AO,-11.202692,17.873887,Angola,False,True,False,False,False,True,False,False,True,True
9,AQ,-75.250973,-0.071389,Antarctica,False,False,False,False,False,False,False,False,False,False


In [156]:
print("EU Countries: {}".format(len(coordinates[coordinates['in_eu'] == True])))
print("UNCFCC Countries: {}".format(len(coordinates[coordinates['in_uncfcc'] == True])))
print("nato Countries: {}".format(len(coordinates[coordinates['in_nato'] == True])))
print("oecd Countries: {}".format(len(coordinates[coordinates['in_oecd'] == True])))
print("g7 Countries: {}".format(len(coordinates[coordinates['in_g7'] == True])))
print("UN Countries: {}".format(len(coordinates[coordinates['in_un'] == True])))
print("OSCE Countries: {}".format(len(coordinates[coordinates['in_osce'] == True])))
print("COE Countries: {}".format(len(coordinates[coordinates['in_coe'] == True])))
print("ILO Countries: {}".format(len(coordinates[coordinates['in_ilo'] == True])))
print("Interpol Countries: {}".format(len(coordinates[coordinates['in_interpol'] == True])))

EU Countries: 27
UNCFCC Countries: 187
nato Countries: 28
oecd Countries: 37
g7 Countries: 7
UN Countries: 169
OSCE Countries: 56
COE Countries: 46
ILO Countries: 164
Interpol Countries: 182


In [155]:
coordinates.to_json(r'data.json', orient="records")

In [154]:
len(coordinates)

245